In [855]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, RFE, SelectFromModel, SelectKBest, f_classif, chi2, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.datasets import load_iris

In [ ]:
# motivation of feature selection
# Adding more features make the model more complex, and the model may overfit the data. By removing the unimportant features,
# the model may generalize better. 
# this totorial is based on the material on sklearn website and other resources. Please see the reference at the end.

### get the data 

In [904]:
# Read the data, for more information of this data: https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html
iris = load_iris()
X = iris.data
y = iris.target

In [905]:
print(X[: 5, :])
print(X.shape)
# the data have 4 features. 

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
(150, 4)


In [858]:
# In order to test the effectness of different feature selection methods, we add some noise features to the dataset.
np.random.seed(100)
E = np.random.uniform(0, 1, size=(len(X), 10))
X = np.hstack((X, E))
X.shape
# now the data have 14 features

(150, 14)

In [859]:
# Before applying feature selection method, we need to split the data first.
# The reason for this is that we only select features based on the information on the training set, not on the whole data set. 
# We should keep part of the whole data set as test set to evaluate the performance of the feature selection and the model.

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, test_size=0.3)
X_train.shape

(105, 14)

### Removing features with low variance

In [ ]:
# VarianceThreshold is a simple baseline approach to feature selection. It removes all features whose variance
# doesn’t meet some threshold. By default, it removes all zero-variance features, 
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold

In [860]:
sel_variance_threshold = VarianceThreshold() 
X_train_remove_variance = sel_variance_threshold.fit_transform(X_train)
X_train_remove_variance.shape
# The data still has 14 features, none of the features were removed

(105, 14)

### Univariate feature selection

In [ ]:
# Univariate feature selection works by selecting the best features based on univariate statistical tests.
# we compare each feature to the target variable, to see whether there is statistially significant relationship between them.
# It is also called analysis of variance (ANOVA). 
# When we analyze the relationship between one feature and the target variable  
# we ignore the other features. That is why it is called 'univariate'.
# Each feature has its own test score. 
# Finally, all the test scores are compared, and the features with top scores will be selected. 

In [861]:
# (1) use chi square test. For more information about chi square test, read:
# a.  statistics: http://vassarstats.net/textbook/  , chaper 8
# b.  sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2
# read the source code if you want to know more about how sklearn apply chi squre test: https://github.com/scikit-learn/scikit-learn/blob/1495f6924/sklearn/feature_selection/univariate_selection.py#L172

sel_chi2 = SelectKBest(chi2, k=4) # select 4 features
X_train_chi2 = sel_chi2.fit_transform(X_train, y_train)

print(sel_chi2.get_support())
print(sel_chi2.get_params())

# The first 4 elements in the array are trues, which means the first 4 features were selected by this method.
# Since the first 4 features are the original features in the data, the chi squre test performs well.

[ True  True  True  True False False False False False False False False
 False False]
{'k': 4, 'score_func': <function chi2 at 0x0000022EBF1BE1E0>}


In [862]:
# (2) use f test 
# a. statistics
#   one-way ANOVA : http://vassarstats.net/textbook/  , chaper 14
#   ANOVA for regression: http://facweb.cs.depaul.edu/sjost/csc423/documents/f-test-reg.htm
# b.  sklearn
# f test for classification: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif
# f test for regression: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression

sel_f = SelectKBest(f_classif, k=4)
X_train_f = sel_f.fit_transform(X_train, y_train)

print(sel_f.get_support())
print(sel_f.get_params())

# The f test can correctly select the original features

[ True  True  True  True False False False False False False False False
 False False]
{'k': 4, 'score_func': <function f_classif at 0x0000022EBF1BE0D0>}


In [863]:
# (2) use mutual_info_classif test 
# sklearn
# for classification: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif
# for regression: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html#sklearn.feature_selection.mutual_info_regression

sel_mutual = SelectKBest(mutual_info_classif, k=4)
X_train_mutual = sel_mutual.fit_transform(X_train, y_train)

print(sel_mutual.get_support())
print(sel_mutual.get_params())

# The mutual_info_classif test can correctly select the original features

[ True  True  True  True False False False False False False False False
 False False]
{'k': 4, 'score_func': <function mutual_info_classif at 0x0000022EBF280730>}


In [ ]:
# In sum, three tests produce the same result. 
# We use the iris data as a classification problem. For regression probelm, similarly, we can use f_regression, mutual_info_regression 


### Recursive feature elimination

In [ ]:
# Given an external estimator that assigns weights to features (e.g., the coefficients of a linear model), 
# recursive feature elimination (RFE) is to select features by recursively considering smaller and smaller sets of features. 
# First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through 
#a coef_ attribute or through a feature_importances_ attribute. Then, the least important features are pruned from current 
# set of features.That procedure is recursively repeated on the pruned set until the desired number of features to select 
# is eventually reached.

In [898]:
# use the logistic regresssion as the model
# about RFE in sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE

model_logistic = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000)
sel_rfe_logistic = RFE(estimator=model_logistic, n_features_to_select=4, step=1)
X_train_rfe_logistic = sel_rfe_logistic.fit_transform(X_train, y_train)

In [899]:
print(sel_rfe_logistic.get_support())
print(sel_rfe_logistic.get_params())

[False  True  True  True False False False False False False False False
 False  True]
{'estimator__C': 1.0, 'estimator__class_weight': None, 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__l1_ratio': None, 'estimator__max_iter': 1000, 'estimator__multi_class': 'multinomial', 'estimator__n_jobs': None, 'estimator__penalty': 'l2', 'estimator__random_state': None, 'estimator__solver': 'lbfgs', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), 'n_features_to_select': 4, 'step': 1, 'verbose': 0}


In [895]:
sel_rfe_logistic.ranking_
# The features were selected will all be rank 1.

# The recursive feature elimination select the only part of the original features and 1 noise feature.

array([ 3,  1,  1,  1,  9,  8,  7,  6,  5,  4,  2, 11, 10,  1])

In [896]:
# lets try another model: the random forest

model_tree = RandomForestClassifier(random_state=100, n_estimators=50)
sel_rfe_tree = RFE(estimator=model_tree, n_features_to_select=4, step=1)
X_train_rfe_tree = sel_rfe_tree.fit_transform(X_train, y_train)

In [900]:
print(sel_rfe_tree.get_support())
print(sel_rfe_tree.get_params())

[ True  True  True  True False False False False False False False False
 False False]
{'estimator__bootstrap': True, 'estimator__class_weight': None, 'estimator__criterion': 'gini', 'estimator__max_depth': None, 'estimator__max_features': 'auto', 'estimator__max_leaf_nodes': None, 'estimator__min_impurity_decrease': 0.0, 'estimator__min_impurity_split': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'estimator__min_weight_fraction_leaf': 0.0, 'estimator__n_estimators': 50, 'estimator__n_jobs': None, 'estimator__oob_score': False, 'estimator__random_state': 100, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fra

In [ ]:
# using randome foreast as the model in the recursive feature selection can select the right features in this case

### Feature selection using SelectFromModel

In [ ]:
# SelectFromModel is a meta-transformer that can be used along with any estimator that has a coef_ or feature_importances_ 
# attribute after fitting. The features are considered unimportant and removed, 
# if the corresponding coef_ or feature_importances_ values are below the provided threshold parameter. 
# compared to univariate feature selection, model-based feature selection consider all feature at once, thus can capture interactions
# the model used for the feature selection dosen't need to be the same model for traning later.
# sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel

#### (1) L1-based feature selection

In [ ]:
# linear model with L1 penalty can eliminate some of the features, 
# thus can act as feature selection method before using another model to fit the data

In [867]:
model_logistic = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=10000, penalty='l1')
sel_model_logistic = SelectFromModel(estimator=model_logistic)
X_train_sfm_l1 = sel_model_logistic.fit_transform(X_train, y_train)

In [868]:
print(sel_model_logistic.get_support())
print(sel_model_logistic.get_params())

# The SelectFromModel select the wrong features. Feature 2 in the original dataset was ignored
# while one feature from the noice was selected

[ True False  True  True False False False False False  True False False
 False False]
{'estimator__C': 1.0, 'estimator__class_weight': None, 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__l1_ratio': None, 'estimator__max_iter': 10000, 'estimator__multi_class': 'multinomial', 'estimator__n_jobs': None, 'estimator__penalty': 'l1', 'estimator__random_state': None, 'estimator__solver': 'saga', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='multinomial', n_jobs=None, penalty='l1',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False), 'max_features': None, 'norm_order': 1, 'prefit': False, 'threshold': None}


#### (2)  Tree-based feature selection

In [891]:
# let's try another model
model_tree = RandomForestClassifier(random_state=100, n_estimators=50)
model_tree.fit(X_train, y_train)
print(model_tree.feature_importances_)

sel_model_tree = SelectFromModel(estimator=model_tree, prefit=True, threshold='mean')  
                    # since we already fit the data, we specify prefit option here
                    # Features whose importance is greater or equal to the threshold are kept while the others are discarded.
X_train_sfm_tree = sel_model_tree.transform(X_train)

[0.11844633 0.09434048 0.21340848 0.33708242 0.02019553 0.03081254
 0.02317242 0.01962394 0.02407251 0.02193159 0.03289007 0.01836624
 0.01811144 0.027546  ]


In [892]:
print(sel_model_tree.get_support())
print(sel_model_tree.get_params())

# Based on the tree model, the SelectFromModel select the correct features.

[ True  True  True  True False False False False False False False False
 False False]
{'estimator__bootstrap': True, 'estimator__class_weight': None, 'estimator__criterion': 'gini', 'estimator__max_depth': None, 'estimator__max_features': 'auto', 'estimator__max_leaf_nodes': None, 'estimator__min_impurity_decrease': 0.0, 'estimator__min_impurity_split': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'estimator__min_weight_fraction_leaf': 0.0, 'estimator__n_estimators': 50, 'estimator__n_jobs': None, 'estimator__oob_score': False, 'estimator__random_state': 100, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fra

In [871]:
# In summary: for this particular data set,  While using logistic model as recursive feature elimination or model selection, 
# it select the features incorretly. On the other hand, all other feature selection methods 
# select the first four features correctly.

### Let's compare the performance before and after the feature selection

####  (1) Before feature selection

In [872]:
model_logistic = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=10000)
model_logistic.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [873]:
predict = model_logistic.predict(X_test)
print(confusion_matrix(y_test, predict))
print(classification_report(y_test, predict))

[[16  0  0]
 [ 0 11  0]
 [ 0  2 16]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.85      1.00      0.92        11
           2       1.00      0.89      0.94        18

    accuracy                           0.96        45
   macro avg       0.95      0.96      0.95        45
weighted avg       0.96      0.96      0.96        45



#### (2) After feature selection

In [874]:
# we use the result from the feature selection based on the chi squre test
# X_train_chi2 is the data after the feature selection to feed into the model

model_logistic = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=10000)
model_logistic.fit(X_train_chi2, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [880]:
# We also need to transform the test data because the number of features were changed

X_test_chi2 = sel_chi2.transform(X_test)
print(X_test.shape)
print(X_test_chi2.shape)

# Only use the features in the test set that are corresponding to the remaining features in the training set. 4 features in this case

(45, 14)
(45, 4)


In [881]:
predict = model_logistic.predict(X_test_chi2)
print(confusion_matrix(y_test, predict))
print(classification_report(y_test, predict))

[[16  0  0]
 [ 0 11  0]
 [ 0  1 17]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.92      1.00      0.96        11
           2       1.00      0.94      0.97        18

    accuracy                           0.98        45
   macro avg       0.97      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



In [877]:
# In sum, feature selection remove the noice and generalize the model better, thus improve the model performance

E = np.random.uniform(0, 1, size=(len(X), 10))


In [879]:
Before

[[16  0  0]
 [ 0 11  0]
 [ 0  2 16]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.85      1.00      0.92        11
           2       1.00      0.89      0.94        18

    accuracy                           0.96        45
   macro avg       0.95      0.96      0.95        45
weighted avg       0.96      0.96      0.96        45

After 
[[16  0  0]
 [ 0 11  0]
 [ 0  1 17]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.92      1.00      0.96        11
           2       1.00      0.94      0.97        18

    accuracy                           0.98        45
   macro avg       0.97      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)

In [ ]:
# We can experiment on adding different noise. for example: E = np.random.uniform(0, 10, size=(len(X), 10))
# if the number of features in the noice increase to 20 and the noice is bigger, then the feature selection might select
# the wrong features. However, the performance is still improved!

E = np.random.uniform(0, 10, size=(len(X), 20))

Before 

[[16  0  0]
 [ 0 10  1]
 [ 0  4 14]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.71      0.91      0.80        11
           2       0.93      0.78      0.85        18

    accuracy                           0.89        45
   macro avg       0.88      0.90      0.88        45
weighted avg       0.90      0.89      0.89        45

After 
[[16  0  0]
 [ 0 10  1]
 [ 0  1 17]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.91      0.91      0.91        11
           2       0.94      0.94      0.94        18

    accuracy                           0.96        45
   macro avg       0.95      0.95      0.95        45
weighted avg       0.96      0.96      0.96        45